# **Launch Sites Locations Analysis with Folium**

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.

## Objectives

This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.

Let's first import required Python packages for this lab:


## Import Libraries

In [1]:
#!pip3 install folium
!pip install folium==0.7.0
!pip3 install wget

In [2]:
# folium makes it easy to visualize data that’s been manipulated in Python on an interactive leaflet map. 
# It enables both the binding of data to a map for choropleth visualizations as well as passing rich 
# vector/raster/HTML visualizations as markers on the map.
import folium
print('Folium installed and imported!')

# Wget is a non-interactive utility to download remote files from the internet.
import wget
print('wget installed and imported!')

# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
print('Pandas installed and imported!')

Folium installed and imported!
wget installed and imported!
Pandas installed and imported!


In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
print('MarkerCluster installed and imported!')

# Import folium MousePosition plugin
from folium.plugins import MousePosition
print('MousePosition installed and imported!')

# Import folium DivIcon plugin
from folium.features import DivIcon
print('DivIcon installed and imported!')

MarkerCluster installed and imported!
MousePosition installed and imported!
DivIcon installed and imported!


If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates

The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [5]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [6]:
spacex_df.shape

(56, 13)

In [7]:
spacex_df.columns.to_list()

['Flight Number',
 'Date',
 'Time (UTC)',
 'Booster Version',
 'Launch Site',
 'Payload',
 'Payload Mass (kg)',
 'Orbit',
 'Customer',
 'Landing Outcome',
 'class',
 'Lat',
 'Long']

Now, you can take a look at what are the coordinates for each site.

In [8]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [9]:
launch_sites_df.head()

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [10]:
launch_sites_df.shape

(4, 3)

In [11]:
launch_sites_df.columns.to_list()

['Launch Site', 'Lat', 'Long']

Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [12]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [13]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name

circle = folium.Circle(
    nasa_coordinate, 
    radius=1000, 
    color='#0000FF', 
    fill=True).add_child(folium.Popup('NASA Johnson Space Center')
)

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(100,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small blue circle near the city of Houston and you can zoom-in to see a larger circle.

Now, let's add a circle for each launch site in data frame `launch_sites`

<h3>TODO:</h3> Create and add folium.Circle and folium.Marker for each launch site on the site map

An example of folium.Circle:

`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`

An example of folium.Marker:

`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


<h2>Folium documentation</h2>

https://python-visualization.github.io/folium/modules.html

In [14]:
for launchSite, lat, lng, in zip(launch_sites_df['Launch Site'], launch_sites_df['Lat'], launch_sites_df['Long']):
    print("launchSite = ", launchSite, "\tLatitude = ", lat, "\tLongitude = ", lng,)

launchSite =  CCAFS LC-40 	Latitude =  28.56230197 	Longitude =  -80.57735648
launchSite =  CCAFS SLC-40 	Latitude =  28.56319718 	Longitude =  -80.57682003
launchSite =  KSC LC-39A 	Latitude =  28.57325457 	Longitude =  -80.64689529
launchSite =  VAFB SLC-4E 	Latitude =  34.63283416 	Longitude =  -120.6107455


In [15]:
colors =['red', 'green', 'orange', 'blue']
i = 0

# loop through the 4 launch sites and add each to the sitemap feature group
for launchSite, lat, lng, in zip(launch_sites_df['Launch Site'], launch_sites_df['Lat'], launch_sites_df['Long']):
    # instantiate a feature group for the Launch Sites in the dataframe

    circle = folium.Circle(
        [lat, lng],
        radius=300, # define how big you want the circle markers to be
        color=colors[i],
        fill=True,
        fill_color=colors[i],
        fill_opacity=0.6).add_child(folium.Popup(launchSite)
    )    
    # Create a marker
    marker = folium.map.Marker(
        [lat, lng],
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(100,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launchSite,
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)
    i +=1

print('Concluido')
site_map

Concluido


The generated map with marked launch sites should look similar to the following:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png" />
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?   YES
*   Are all launch sites in very close proximity to the coast?   YES

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [16]:
spacex_df.head(10)

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0
5,CCAFS LC-40,28.562302,-80.577356,0
6,CCAFS LC-40,28.562302,-80.577356,0
7,CCAFS LC-40,28.562302,-80.577356,0
8,CCAFS LC-40,28.562302,-80.577356,0
9,CCAFS LC-40,28.562302,-80.577356,0


In [17]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

Let's first create a `MarkerCluster` object

In [18]:
marker_cluster = MarkerCluster()

<h3>TODO:</h3> Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value

In [19]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red


In [20]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


<h3>TODO:</h3> For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


<h3>Pandas iterrows()</h3>

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iterrows.html

DataFrame.iterrows()   =>  Iterate over DataFrame rows as (index, Series) pairs.

Because iterrows returns a Series for each row, it does not preserve dtypes across the rows (dtypes are preserved across columns for DataFrames). For example,

In [21]:
df = pd.DataFrame([[1, 1.5]], columns=['int', 'float'])
row = next(df.iterrows())[1]
row

int      1.0
float    1.5
Name: 0, dtype: float64

In [22]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame create a Marker object with:
#    -its coordinate
#    -and customize the Marker's icon property to indicate if this launch was successed or failed, 
#          e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']

for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker = folium.map.Marker(
        [record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png" />
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [23]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:



In [24]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

<h3>TODO:</h3> Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [25]:
# find coordinate of the closest coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [26]:
print("launch_sites_df.loc[0,'Launch Site'] = ", launch_sites_df.loc[0,'Launch Site'])
print("launch_sites_df.loc[0,'Long'] = ", launch_sites_df.loc[0,'Long'])
print("launch_sites_df.loc[0,'Lat'] = ", launch_sites_df.loc[0,'Lat'])

launch_sites_df.loc[0,'Launch Site'] =  CCAFS LC-40
launch_sites_df.loc[0,'Long'] =  -80.57735648
launch_sites_df.loc[0,'Lat'] =  28.56230197


In [27]:
print("launch_sites_df.loc[1,'Launch Site'] = ", launch_sites_df.loc[1,'Launch Site'])
print("launch_sites_df.loc[1,'Long'] = ", launch_sites_df.loc[1,'Long'])
print("launch_sites_df.loc[1,'Lat'] = ", launch_sites_df.loc[1,'Lat'])

launch_sites_df.loc[1,'Launch Site'] =  CCAFS SLC-40
launch_sites_df.loc[1,'Long'] =  -80.57682003
launch_sites_df.loc[1,'Lat'] =  28.56319718


In [28]:
print("launch_sites_df.loc[2,'Launch Site'] = ", launch_sites_df.loc[2,'Launch Site'])
print("launch_sites_df.loc[2,'Long'] = ", launch_sites_df.loc[2,'Long'])
print("launch_sites_df.loc[2,'Lat'] = ", launch_sites_df.loc[2,'Lat'])

launch_sites_df.loc[2,'Launch Site'] =  KSC LC-39A
launch_sites_df.loc[2,'Long'] =  -80.64689529
launch_sites_df.loc[2,'Lat'] =  28.57325457


In [29]:
KSC_LC39A_coordinates = [launch_sites_df.loc[2,'Lat'], launch_sites_df.loc[2,'Long']]
CCAFS_LC40_coordinates = [launch_sites_df.loc[0,'Lat'], launch_sites_df.loc[0,'Long']]

KSC_LC39A_2_highway_coordinates = [28.57261, -80.65553]
CCAFS_LC40_2_coast_coordinates = [28.5623, -80.56779]
KSC_LC39A_2_highway_distance = calculate_distance(28.57261, -80.65553,launch_sites_df.loc[2,'Lat'], launch_sites_df.loc[2,'Long'])
CCAFS_LC40_2_coast_distance = calculate_distance(28.5623, -80.56779, launch_sites_df.loc[0,'Lat'], launch_sites_df.loc[0,'Long'])

In [30]:
CCAFS_LC40_2_coast_distance

0.9345773179391524

<h3>TODO:</h3> After obtained its coordinate, create a folium.Marker to show the distance

In [31]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_coast_marker = folium.Marker(
    CCAFS_LC40_2_coast_coordinates,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(CCAFS_LC40_2_coast_distance),
    )
)
site_map.add_child(distance_coast_marker)


In [32]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)

coordinates = [CCAFS_LC40_coordinates,CCAFS_LC40_2_coast_coordinates]
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png" />
</center>


<h3>TODO:</h3> Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first

A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png" />
</center>


A highway map symbol may look like this:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png" />
</center>


A city map symbol may look like this:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png" />
</center>


<h4>Closest City</h4>

* Closest city => Melbourne FL
* Closest highway = Kennedy Parkway North

In [33]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

In [34]:
melbourne_city_coordinates = [28.118488, -80.670372]
highway_coordinates = [28.57058, -80.65536]
railway_coordinates = [28.57163, -80.58515]

In [35]:
CCAFS_LC40_2_Melbourne_distance = calculate_distance(28.118488, -80.670372, launch_sites_df.loc[0,'Lat'], launch_sites_df.loc[0,'Long'])
CCAFS_LC40_2_highway_distance = calculate_distance(28.57058, -80.65536, launch_sites_df.loc[0,'Lat'], launch_sites_df.loc[0,'Long'])
CCAFS_LC40_2_railway_distance = calculate_distance(28.57163, --80.58515, launch_sites_df.loc[0,'Lat'], launch_sites_df.loc[0,'Long'])
print('distance_city =',CCAFS_LC40_2_Melbourne_distance, ' km')
print('distance_highway =',CCAFS_LC40_2_highway_distance, ' km')
print('distance_railway =',CCAFS_LC40_2_railway_distance, ' km')

distance_city = 50.19818198626525  km
distance_highway = 7.675519207785525  km
distance_railway = 13357.613329063079  km


In [36]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_Melbourne_marker = folium.Marker(
    melbourne_city_coordinates,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(CCAFS_LC40_2_Melbourne_distance),
    )
)
site_map.add_child(distance_Melbourne_marker)

distance_Highway_marker = folium.Marker(
    highway_coordinates,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(CCAFS_LC40_2_highway_distance),
    )
)
site_map.add_child(distance_Highway_marker)

distance_Railway_marker = folium.Marker(
    railway_coordinates,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(CCAFS_LC40_2_railway_distance)
    )
)
site_map.add_child(distance_Railway_marker)


In [37]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)

coordinates01 = [CCAFS_LC40_coordinates,melbourne_city_coordinates]
lines01=folium.PolyLine(locations=coordinates01, weight=1)
site_map.add_child(lines01)

coordinates02 = [CCAFS_LC40_coordinates,highway_coordinates]
lines02=folium.PolyLine(locations=coordinates01, weight=1)
site_map.add_child(lines02)

coordinates03 = [CCAFS_LC40_coordinates,railway_coordinates]
lines03=folium.PolyLine(locations=coordinates01, weight=1)
site_map.add_child(lines03)

In [38]:
site_map